# Installation

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.8 MB/s 
     |████████████████████████████████| 441 kB 57.6 MB/s 
     |████████████████████████████████| 212 kB 72.0 MB/s 
     |████████████████████████████████| 163 kB 66.2 MB/s 
     |████████████████████████████████| 115 kB 64.8 MB/s 
     |████████████████████████████████| 127 kB 89.7 MB/s 
     |████████████████████████████████| 115 kB 71.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 27.3 MB/s 
     |████████████████████████████████| 7.6 MB 46.2 MB/s 


# Imports

In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (DataCollatorWithPadding,create_optimizer,DebertaTokenizerFast)

In [ ]:
BATCH_SIZE=32
MAX_LENGTH=512

# Data Preparation for Bert Model

In [ ]:
dataset=load_dataset("covid_qa_deepset")
#dataset=load_dataset("squad")

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Dataset covid_qa_deepset downloaded and prepared to /root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 2019
    })
})

In [ ]:
dataset['train'][0]

{'document_id': 630,
 'context': "Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide. Given that the C-type lectin receptor, dendritic cell-specific ICAM-grabbing non-integrin-related (DC-SIGNR, also known as CD209L or liver/lymph node–specific ICAM-grabbing non-integrin (L-SIGN)), can interact with pathogens including HIV-1 and is expressed at the maternal-fetal interface, we hypothesized that it could influence MTCT of HIV-1. METHODS AND FINDINGS: To investigate the potential role of DC-SIGNR in MTCT of HIV-1, we carried out a genetic association study of DC-SIGNR in a

In [ ]:
answer='Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.'
print(len(answer)+370)

465


In [ ]:
# from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# model_id="microsoft/deberta-base"
# tokenizer=DebertaTokenizerFast.from_pretrained(model_id)

In [ ]:
from transformers import LongformerTokenizerFast,TFLongformerForQuestionAnswering
model_id="allenai/longformer-large-4096-finetuned-triviaqa"
tokenizer = LongformerTokenizerFast.from_pretrained(model_id)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/866 [00:00<?, ?B/s]

In [ ]:
tokenized_examples = tokenizer(
    dataset['train'][0]['question'],
    dataset['train'][0]["context"],
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [ ]:
len(tokenized_examples["input_ids"])

18

In [ ]:
list_token=tokenizer.tokenize("[CLS]What is the main cause of HIV-1 infection in children?[SEP]Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child Transmission of HIV-1\n\nhttps://www.ncbi.nlm.nih.gov/pmc/articles/PMC2752805/\n\nBoily-Larouche, Geneviève; Iscache, Anne-Laure; Zijenah, Lynn S.; Humphrey, Jean H.; Mouland, Andrew J.; Ward, Brian J.; Roger, Michel\n2009-10-07\nDOI:10.1371/journal.pone.0007211\nLicense:cc-by\n\nAbstract: BACKGROUND: Mother-to-child transmission (MTCT) is the main cause of HIV-1 infection in children worldwide.")
print(list_token)
for i in range(len(list_token)):
  if list_token[i]=='Ġchildren':
    print(i)

['[CLS]', 'what', 'is', 'the', 'main', 'cause', 'of', 'hiv', '-', '1', 'infection', 'in', 'children', '?', '[SEP]', 'functional', 'genetic', 'variants', 'in', 'dc', '-', 'sign', '##r', 'are', 'associated', 'with', 'mother', '-', 'to', '-', 'child', 'transmission', 'of', 'hiv', '-', '1', 'https', ':', '/', '/', 'www', '.', 'nc', '##bi', '.', 'nl', '##m', '.', 'ni', '##h', '.', 'gov', '/', 'pm', '##c', '/', 'articles', '/', 'pm', '##c', '##27', '##52', '##80', '##5', '/', 'boil', '##y', '-', 'la', '##rou', '##che', ',', 'genevieve', ';', 'is', '##ca', '##che', ',', 'anne', '-', 'lau', '##re', ';', 'z', '##ij', '##ena', '##h', ',', 'lynn', 's', '.', ';', 'humphrey', ',', 'jean', 'h', '.', ';', 'mo', '##ula', '##nd', ',', 'andrew', 'j', '.', ';', 'ward', ',', 'brian', 'j', '.', ';', 'roger', ',', 'michel', '2009', '-', '10', '-', '07', 'doi', ':', '10', '.', '137', '##1', '/', 'journal', '.', 'po', '##ne', '.', '000', '##7', '##21', '##1', 'license', ':', 'cc', '-', 'by', 'abstract', ':', 

In [ ]:
tokenizer.encode("What is the main cause of HIV-1 infection in children?")

[101,
 2054,
 2003,
 1996,
 2364,
 3426,
 1997,
 9820,
 1011,
 1015,
 8985,
 1999,
 2336,
 1029,
 102]

In [ ]:
for ids in tokenized_examples["input_ids"]:
  print(ids)
  print('-->',tokenizer.decode(ids))
  #break

[101, 2054, 2003, 1996, 2364, 3426, 1997, 9820, 1011, 1015, 8985, 1999, 2336, 1029, 102, 8360, 7403, 10176, 1999, 5887, 1011, 3696, 2099, 2024, 3378, 2007, 2388, 1011, 2000, 1011, 2775, 6726, 1997, 9820, 1011, 1015, 16770, 1024, 1013, 1013, 7479, 1012, 13316, 5638, 1012, 17953, 2213, 1012, 9152, 2232, 1012, 18079, 1013, 7610, 2278, 1013, 4790, 1013, 7610, 2278, 22907, 25746, 17914, 2629, 1013, 26077, 2100, 1011, 2474, 22494, 5403, 1010, 20245, 1025, 2003, 3540, 5403, 1010, 4776, 1011, 21360, 2890, 1025, 1062, 28418, 8189, 2232, 1010, 9399, 1055, 1012, 1025, 15462, 1010, 3744, 1044, 1012, 1025, 9587, 7068, 4859, 1010, 4080, 1046, 1012, 1025, 4829, 1010, 4422, 1046, 1012, 1025, 5074, 1010, 8709, 2268, 1011, 2184, 1011, 5718, 9193, 1024, 2184, 1012, 14989, 2487, 1013, 3485, 1012, 13433, 2638, 1012, 2199, 2581, 17465, 2487, 6105, 1024, 10507, 1011, 2011, 10061, 1024, 4281, 1024, 2388, 1011, 2000, 1011, 2775, 6726, 1006, 11047, 6593, 1007, 2003, 1996, 2364, 3426, 1997, 9820, 1011, 1015, 898

In [ ]:
offset_mapping_list=[(0, 4), (4, 7), (7, 11), (11, 16), (16, 22), (22, 25), (25, 29), (29, 30), (30, 31), (31, 41), (41, 44), (44, 53), (53, 54), (0, 0), (0, 8), (8, 10), (10, 18), (18, 23), (23, 27), (27, 30), (30, 33), (33, 34), (34, 38), (38, 39), (39, 43), (43, 54), (54, 59), (59, 66), (66, 67), (67, 69), (69, 70), (70, 75), (75, 88), (88, 91), (91, 95), (95, 96), (96, 97), (97, 98), (98, 99), (99, 104), (104, 107), (107, 110), (110, 111), (111, 113), (113, 115), (115, 116), (116, 118), (118, 119), (119, 120), (120, 123), (123, 124), (124, 127), (127, 128), (128, 130), (130, 131), (131, 132), (132, 140), (140, 141), (141, 143), (143, 144), (144, 146), (146, 148), (148, 151), (151, 152), (152, 153), (153, 154), (154, 156), (156, 159), (159, 160), (160, 161), (161, 163), (163, 165), (165, 168), (168, 169), (169, 174), (174, 176), (176, 177), (177, 179), (179, 180), (180, 183), (183, 188), (188, 189), (189, 194), (194, 195), (195, 197), (197, 200), (200, 201), (201, 203), (203, 205), (205, 207), (207, 209), (209, 210), (210, 215), (215, 217), (217, 219), (219, 225), (225, 228), (228, 229), (229, 234), (234, 236), (236, 238), (238, 242), (242, 246), (246, 247), (247, 254), (254, 256), (256, 258), (258, 263), (263, 264), (264, 270), (270, 272), (272, 274), (274, 280), (280, 281), (281, 288), (288, 289), (289, 293), (293, 294), (294, 296), (296, 297), (297, 299), (299, 300), (300, 302), (302, 303), (303, 304), (304, 306), (306, 307), (307, 309), (309, 311), (311, 312), (312, 319), (319, 320), (320, 321), (321, 324), (324, 325), (325, 328), (328, 330), (330, 332), (332, 333), (333, 340), (340, 341), (341, 343), (343, 344), (344, 346), (346, 347), (347, 348), (348, 356), (356, 357), (357, 362), (362, 368), (368, 369), (369, 376),]
print(len(offset_mapping_list))

152


In [ ]:
tokenized_examples

{'input_ids': [[101, 2054, 2003, 1996, 2364, 3426, 1997, 9820, 1011, 1015, 8985, 1999, 2336, 1029, 102, 8360, 7403, 10176, 1999, 5887, 1011, 3696, 2099, 2024, 3378, 2007, 2388, 1011, 2000, 1011, 2775, 6726, 1997, 9820, 1011, 1015, 16770, 1024, 1013, 1013, 7479, 1012, 13316, 5638, 1012, 17953, 2213, 1012, 9152, 2232, 1012, 18079, 1013, 7610, 2278, 1013, 4790, 1013, 7610, 2278, 22907, 25746, 17914, 2629, 1013, 26077, 2100, 1011, 2474, 22494, 5403, 1010, 20245, 1025, 2003, 3540, 5403, 1010, 4776, 1011, 21360, 2890, 1025, 1062, 28418, 8189, 2232, 1010, 9399, 1055, 1012, 1025, 15462, 1010, 3744, 1044, 1012, 1025, 9587, 7068, 4859, 1010, 4080, 1046, 1012, 1025, 4829, 1010, 4422, 1046, 1012, 1025, 5074, 1010, 8709, 2268, 1011, 2184, 1011, 5718, 9193, 1024, 2184, 1012, 14989, 2487, 1013, 3485, 1012, 13433, 2638, 1012, 2199, 2581, 17465, 2487, 6105, 1024, 10507, 1011, 2011, 10061, 1024, 4281, 1024, 2388, 1011, 2000, 1011, 2775, 6726, 1006, 11047, 6593, 1007, 2003, 1996, 2364, 3426, 1997, 9820, 

In [ ]:
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,1,1,1,2,2,2,2,2,3,3,3,...

In [ ]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
offset_mapping = tokenized_examples.pop("offset_mapping")

tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
for i, offsets in enumerate(offset_mapping):
  if len(dataset['train'][0]['answers']['answer_start'])==0:
    tokenized_examples["start_positions"].append(0)
    tokenized_examples["end_positions"].append(0)
  else:
      
    start_char=dataset['train'][0]['answers']['answer_start'][0]
    end_char=start_char+len(dataset['train'][0]['answers']['text'][0])
    found=0
    start_token_position=0
    end_token_position=0
    
    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH
        
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

In [ ]:
print(tokenized_examples['start_positions'])
print(tokenized_examples['end_positions'])

[145, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[167, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(tokenized_examples['input_ids'])

18


In [ ]:
def preprocess_function(dataset):
  
  questions = [q.lstrip() for q in dataset["question"]]
  paragraphs = [p.lstrip() for p in dataset["context"]]
  
  tokenized_examples = tokenizer(
    questions,
    paragraphs,
    truncation="only_second",
    max_length=MAX_LENGTH,
    stride=64,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples["start_positions"] = []#[152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  tokenized_examples["end_positions"] = []#[172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

  for i, offsets in enumerate(offset_mapping):
    sample_index = sample_mapping[i]

    start_char=dataset["answers"][sample_index]['answer_start'][0]
    end_char=start_char+len(dataset["answers"][sample_index]['text'][0])
    found=0
    start_token_position=0
    end_token_position=0
    
    for j,offset in enumerate(offsets):
      if offset[0]<=start_char and offset[1]>=start_char and found==0:
        start_token_position=j
        end_token_position=MAX_LENGTH
        found=1
      if offset[1]>=end_char and found==1:
        end_token_position=j
        break
    tokenized_examples["start_positions"].append(start_token_position)
    tokenized_examples["end_positions"].append(end_token_position)

  return tokenized_examples

In [ ]:
tokenized_dataset=dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 4537
    })
})

In [ ]:
tf_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle=True, 
    batch_size=BATCH_SIZE,
)

In [ ]:
for i in tf_dataset.take(10):
  print(i)

{'input_ids': <tf.Tensor: shape=(4, 4096), dtype=int64, numpy=
array([[    0,  3750,    99, ...,     1,     1,     1],
       [    0,  2264,    16, ...,    50, 35993,     2],
       [    0,  2264, 19729, ..., 18225,     9,     2],
       [    0,  2264, 27460, ...,     9,   208,     2]])>, 'attention_mask': <tf.Tensor: shape=(4, 4096), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>, 'start_positions': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([  0, 425,   0,   0])>, 'end_positions': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([  0, 475,   0,   0])>}
{'input_ids': <tf.Tensor: shape=(4, 4096), dtype=int64, numpy=
array([[    0,  2264,   473, ...,   731,  5457,     2],
       [    0,  2264,    16, ...,     1,     1,     1],
       [    0,  2264,  3164, ..., 41675,  9937,     2],
       [    0,  2264,    16, ...,   112, 37109,     2]])>, 'attention_mask': <tf.Tensor: shape=(4, 

In [ ]:
train_dataset=tf_dataset.take(int(0.9*len(tf_dataset)))

In [ ]:
val_dataset=tf_dataset.skip(int(0.9*len(tf_dataset)))

# Modeling

In [ ]:
from transformers import LongformerTokenizer, TFLongformerForQuestionAnswering

model = TFLongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

Downloading:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFLongformerForQuestionAnswering.

All the layers of TFLongformerForQuestionAnswering were initialized from the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerForQuestionAnswering for predictions without further training.


' ELISA'

In [ ]:
model.summary()

Model: "tf_longformer_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 longformer (TFLongformerMai  multiple                 433551360 
 nLayer)                                                         
                                                                 
 qa_outputs (Dense)          multiple                  2050      
                                                                 
Total params: 433,553,410
Trainable params: 433,553,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer=Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
history=model.fit(train_dataset,validation_data=val_dataset,epochs=3)

Epoch 1/3


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


There should be exactly three separator tokens: 2 in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this. This is most likely an error. The global attention is disabled for this forward pass.


ResourceExhaustedError: ignored

# Evaluation

In [ ]:
from evaluate import load

In [ ]:
squad_metric = load("squad")
predictions = [{'prediction_text': '1999', 'id': '56e10a3be3433e1400422b22'}]
references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
results = squad_metric.compute(predictions=predictions, references=references)
print(results)

{'exact_match': 0.0, 'f1': 0.0}


# Testing

In [ ]:
question="How is the virus spread?"
text="We know that the disease is caused by the SARS-CoV-2 virus, which spreads between people in several different ways.Current evidence suggests that the virus spreads mainly between people who are in close contact with each other, for example at a conversational distance.The virus can spread from an infected person’s mouth or nose in small liquid particles when they cough, sneeze, speak, sing or breathe. Another person can then contract the virus when infectious particles that pass through the air are inhaled at short range (this is often called short-range aerosol or short-range airborne transmission) or if infectious particles come into direct contact with the eyes, nose, or mouth (droplet transmission). The virus can also spread in poorly ventilated and/or crowded indoor settings, where people tend to spend longer periods of time. This is because aerosols can remain suspended in the air or travel farther than conversational distance (this is often called long-range aerosol or long-range airborne transmission). People may also become infected when touching their eyes, nose or mouth after touching surfaces or objects that have been contaminated by the virus. Further research is ongoing to better understand the spread of the virus and which settings are most risky and why. Research is also under way to study virus variants that are emerging and why some are more transmissible. For updated information on SARS-CoV-2 variants, please read the weekly epidemiologic updates."
inputs = tokenizer(question, text, return_tensors="tf")
outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

' mouth'